# Import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import joblib
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_curve
from sklearn.metrics import confusion_matrix, classification_report, auc, precision_recall_curve, average_precision_score
import torch
import keras

# Check GPU/CUDA

In [ ]:

print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Number:", torch.cuda.device_count())
    print("Current GPU Index:", torch.cuda.current_device())
    print("GPU Type:", torch.cuda.get_device_name(device=None))
    print("GPU Capability:", torch.cuda.get_device_capability(device=None))
    print("Is GPU Initialized yet?", torch.cuda.is_initialized())

# Load data

In [ ]:
sensor = "Landsat"
sat_comb = "Hypso_filter"
hypso_comb = "all_filtered_reflectance"
dr_type = "original"

dir_name = f"{sensor}/{sat_comb}/{dr_type}"

In [ ]:
# Load existing classification reports if the file exists
classification_reports_path = f"{dir_name}/classification_reports.json"
if os.path.exists(classification_reports_path):
    with open(classification_reports_path, "r") as json_file:
        classification_reports = json.load(json_file)
else:
    classification_reports = {}

model_version = classification_reports[0][0]
print(model_version)

In [ ]:
X_train = np.load(f"./data/{hypso_comb}_X_train_{sensor}_{sat_comb}.npy")
X_test = np.load(f"./data/{hypso_comb}_X_test_{sensor}_{sat_comb}.npy")
y_train = np.load(f"./data/{hypso_comb}_y_train_{dr_type}.npy")
y_test = np.load(f"./data/{hypso_comb}_y_test_{dr_type}.npy")

In [ ]:
def one_hot_encoding(data):
  L_E = LabelEncoder()
  integer_encoded = L_E.fit_transform(data)  
  onehot_encoder = OneHotEncoder(sparse_output=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  one_hot_encoded_data = onehot_encoder.fit_transform(integer_encoded)
  return one_hot_encoded_data

y_train_encoded = one_hot_encoding(y_train.ravel())
y_test_encoded = one_hot_encoding(y_test.ravel())


print(y_train_encoded.shape)
print(y_test_encoded.shape)

# Train model

In [ ]:
model_version = classification_reports[0][0] 
model = tf.keras.models.load_model(f"{dir_name}/{model_version}.keras")
initial_epochs = 50
fine_tune_epochs = 20
gamma = 4
learning_rate = 1e-5
trainable_layer_start_idx = -54

save_model_version = classification_reports[0][0] + f"_tl_ie{initial_epochs}_fte{fine_tune_epochs}_g{gamma}_lr{learning_rate}_tbl{trainable_layer_start_idx}"

# model.summary()

initial_epochs = 50

# Freeze middle layers only
for layer in model.layers[:-5]:  # Adjust indices based on your model structure
    layer.trainable = False

model.summary()

# Compile the model
model.compile(loss=keras.losses.CategoricalFocalCrossentropy(gamma=gamma), 
                             optimizer=keras.optimizers.RMSprop(), 
                             metrics=['mse', 'accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, mode='min'), 
             tf.keras.callbacks.ModelCheckpoint(f"{dir_name}/{save_model_version}_tl.keras", verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history = model.fit(X_train, y_train_encoded, epochs=initial_epochs, batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim([0,1])
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylim([0,1])
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Fine-tune model

In [ ]:

# Unfreeze middle layers
for layer in model.layers[trainable_layer_start_idx:]:  # Adjust indices based on your model structure
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True  # Unfreeze non-BatchNorm layers

# model.summary()

fine_tune_epochs = 20
total_epochs =  initial_epochs + fine_tune_epochs

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss=keras.losses.CategoricalFocalCrossentropy(gamma = gamma),
                             metrics=['mse', 'accuracy'])

# Fine-tune the entire model
callbacks_fine = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, mode='min'), 
             tf.keras.callbacks.ModelCheckpoint(f"{dir_name}/{save_model_version}_ft.keras", verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
history_fine = model.fit(X_train, y_train_encoded, epochs=total_epochs, initial_epoch=len(history.epoch), batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Evaluate model

In [ ]:
# Predictions from the Test Set from the Trained Model
base_model = tf.keras.models.load_model(f"{dir_name}/{model_version}.keras")
Predictions_base_model = base_model.predict(X_test, verbose=1)
print(Predictions_base_model.shape)

# Predictions from the Test Set from the Trained Model
best_model_tl = tf.keras.models.load_model(f"{dir_name}/{save_model_version}_tl.keras")
Predictions = best_model_tl.predict(X_test, verbose=1)
print(Predictions.shape)

In [ ]:
# Error of the prediction, one of many evaluation metrics
# Using Mean Absolute Error (MAE) in this case as a sample
Error = mean_absolute_error(y_test_encoded, Predictions_base_model)
print(f"MAE: {Error}")
Error_tl = mean_absolute_error(y_test_encoded, Predictions)
print(f"MAE_tl: {Error_tl}")

In [ ]:
def history_plot(history):
  # list all dictionaries in history
  print(history.history.keys())
  # summarize history for error
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,1)
  plt.plot(history.history['mse'])
  plt.plot(history.history['val_mse'])
  plt.title('Model Error Performance')
  plt.ylabel('Error')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
  # summarize history for loss
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
#
history_plot(history)
history_plot(history_fine)

# Save evaluation

In [ ]:
# Create directory if it does not exist

if not os.path.exists(dir_name):
    os.makedirs(f"{dir_name}", exist_ok=True)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import json
import matplotlib.pyplot as plt



y_pred = np.argmax(Predictions, axis=1) + 1

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1-score: {f1}")

report_dict = classification_report(y_test, y_pred, output_dict=True)

# Load existing classification reports if the file exists
classification_reports_path = f"{dir_name}/classification_reports.json"
if os.path.exists(classification_reports_path):
    with open(classification_reports_path, "r") as json_file:
        classification_reports = json.load(json_file)
else:
    classification_reports = {}

# Add the current model's classification report
classification_reports.append([save_model_version, report_dict])

# Sort the classification reports by macro F1-score
sorted_reports = sorted(
    classification_reports,
    key=lambda x: x[1]["macro avg"]["f1-score"],
    reverse=True
)

# Save the updated classification reports
with open(classification_reports_path, "w") as json_file:
    json.dump(sorted_reports, json_file, indent=4)

class_dict = {
    0: "Spruce",
    1: "Pine",
    3: "Deciduous"
}

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_dict.values(), yticklabels=class_dict.values())
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(f"{dir_name}/{save_model_version}_confusion_matrix.png")
plt.show()